In [2]:
import numpy as np
import plotly.graph_objs as go
import plotly.offline as py

py.init_notebook_mode(connected=True)

%load_ext autoreload
%autoreload 2

In [4]:
peak_cpu = 200 # GFLOPS
peak_mem = 30 # GB/s

ai = 0.25

cutoff_ai = 1.897

log_ai = np.arange(-10, 4.0, 0.1)

log_peak_cpu = [np.log(peak_cpu)] * len(log_ai)
log_peak_mem = log_ai + np.log(peak_mem)

log_ai_peak = np.log(peak_cpu) - np.log(peak_mem)
c1_c2_peak = np.log(ai) + np.log(peak_mem)
layout = go.Layout(
    # title='Roofline Model',
    xaxis=dict(
        title='Arithmetic Intensity [G FLOP/bytes] (log-scale)',
        zeroline=False,
    ),
    yaxis=dict(
        title='Actual G FLOPS (log-scale)',
        zeroline=True,
    ),
    legend=dict(
        x=0.75,
        y=0.075,
    ),
    showlegend=False,
    annotations=[
        dict(
            x=-2,
            y=6,
            xref='x',
            yref='y',
            text=f'Peak FLOPS ({peak_cpu} GFLOP/s)',
            showarrow=False,
            font=dict(
                color='red',
            ),
        ),
        dict(
            x=-4,
            y=1,
            xref='x',
            yref='y',
            text=f'Peak B/w ({peak_mem} GB/s)',
            showarrow=False,
            font=dict(
                color='blue',
            ),
        ),
        dict(
            x=cutoff_ai,
            y=-2,
            xref='x',
            yref='y',
            text='Memory-bound     Compute-bound',
            showarrow=False,
            font=dict(
                color='green',
            ),
        ),
        dict(
            x=cutoff_ai,
            y=-6.6,
            xref='x',
            yref='y',
            text='log(6.66) = '+str(cutoff_ai),
            showarrow=False,
            font=dict(
                color='green',
            ),
        ),
        dict(
            x=np.log(ai),
            y=-6.6,
            xref='x',
            yref='y',
            text = 'log(0.25) = ' + str(round(np.log(ai), 2)),
            showarrow=False,
            font=dict(
                color='gray',
            ),
        ),

    ]
)

cpu_peak_trace = go.Scatter(
    name=f'Peak FLOPS ({peak_cpu} GFLOP/s)',
    x=log_ai,
    y=log_peak_cpu,
    mode='lines',
    line=dict(
        width=1.2,
        color='red'
    ),
)

mem_peak_trace = go.Scatter(
    name=f'Peak B/w ({peak_mem} GB/s)',
    x=log_ai,
    y=log_peak_mem,
    mode='lines',
    line=dict(
        width=1.2,
        color='blue'
    ),
)

bound_range = np.arange(-6, np.log(peak_cpu) + 0.1, 0.1)
mem_compute_bound = go.Scatter(
    name=f'Mem/CPU Limit ({cutoff_ai})',
    x=[log_ai_peak] * len(bound_range),
    y=bound_range,
    mode='lines',
    line=dict(
        width=2,
        color='green',
        dash='dot',
    ),
)

bound_range = np.arange(-6, c1_c2_peak, 0.1)
c1_c2_ai = go.Scatter(
    name=f'Arithmetic Intensity ({ai})',
    x=[np.log(ai)] * len(bound_range),
    y=bound_range,
    mode='lines',
    line=dict(
        width=2,
        color='gray',
        dash='dot',
    ),
)

results = [2181350400, 1763863040, 4709610496, 2756318208, 8600307712, 2687484928, 5384078, 4110728,\
           8185584777, 2627595523]
obs_values = []
bw = [8.72, 7.05, 18.83, 11.02, 34.40, 10.75, 0.02, 0.01, 32.74, 10.51]
for i,flops in enumerate(results):
    obs_values += [go.Scatter(
    name=f'dp{int(1+i//2)}, E_{1+i%2} ',
    x=[np.log(ai)],
    y=[np.log(flops/10**9)],
    text=f'dp{int(1+i//2)}, E{1+i%2} |Bw= {bw[i]} )',
    mode='markers',
    marker=dict(
        size=8,
        showscale=False,
        symbol='x-dot',
    ),
)]

fig = go.Figure(data=[
    cpu_peak_trace, 
    mem_peak_trace, 
    mem_compute_bound,
    c1_c2_ai] + obs_values, layout=layout)
fig.update_xaxes(hoverformat=".3f")
fig.update_yaxes(hoverformat=".3f")
py.iplot(fig, config=dict(showLink=False))
py.offline.plot(fig, filename='roofline.html') 

'roofline.html'